## Predicting a Successful New York Times Article Comment

IPython Process Book

An important part of your project is your iPython process book. Your process book details your steps in developing your solution, including how you collected the data, alternative solutions you tried, describing statistical methods you used, and the insights you got. Equally important to your final results is how you got there! Your process book is the place you describe and document the space of possibilities you explored at each step of your project. We strongly advise you to include many visualizations in your process book.Your process book should include the following topics. Depending on your project type the amount of discussion you devote to each of them will vary:


<strong>Overview and Motivation</strong>: Provide an overview of the project goals and the motivation for it. Consider that this will be read by people who did not see your project proposal.

<strong>Related Work</strong>: Anything that inspired you, such as a paper, a web site, or something we discussed in class.

<strong>Initial Questions</strong>: What questions are you trying to answer? How did these questions evolve over the course of the project? What new questions did you consider in the course of your analysis?

<strong>Data</strong>: Source, scraping method, cleanup, storage, etc.

<strong>Exploratory Data Analysis</strong>: What visualizations did you use to look at your data in different ways? What are the different statistical methods you considered? Justify the decisions you made, and show any major changes to your ideas. How did you reach these conclusions?

<strong>Final Analysis</strong>: What did you learn about the data? How did you answer the questions? How can you justify your answers?

<strong>Presentation</strong>: Present your final results in a compelling and engaging way using text, visualizations, images, and videos on your project web site.

# Overview and Motivation

# Related Work

# Initial Questions

# Data

In [ ]:
# After a few requests, comments_data.content is the generic New York Times error page, but
# comments_data does not have a 404 response or anything. It's a valid request, but NY Times
# can't process it for whatever reason. The rate limit is 30 calls/second and 5000/day, so
# that's not the issue. See if you can get it to work, I guess.
#
# I used simplejson, which you can install with pip, because it gives better error messages,
# but you can just replace simplejson with json and it should work the same.

import requests, time, simplejson, sys
from datetime import date, datetime, timedelta
def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

# Scrape 300 comments per day from Nov. 1, 2014 to Oct. 31, 2015
comments = []
for da in perdelta(date(2014, 11, 1), date(2015, 11, 1), timedelta(days=1)):
    print da
    for i in range(12):
        url = ('http://api.nytimes.com/svc/community/v3/user-content/' +
               'by-date.json?api-key=PUTYOURAPIKEYHERE&date=' + str(da) + # put your API key in here
               '&offset=' + str(25*i))
        comments_data = requests.get(url)
        try:
            data = simplejson.loads(comments_data.content) # error happens here
        except:
            print url, comments_data, comments_data.content
            sys.exit()
        for d in data['results']['comments']:
            comments.append(d)
        time.sleep(2)

# Save the data
with open("comments.json", 'w') as f:
    simplejson.dump(comments, f)

# Uncomment for error-tolerant version
#
# for da in perdelta(date(2014, 11, 5), date(2015, 11, 1), timedelta(days=1)):
#     comments = []
#     print da
#     for i in range(12):
#         success = False
#         url = ('http://api.nytimes.com/svc/community/v3/user-content/' +
#                'by-date.json?api-key=KEY&date=' + str(da) +
#                '&offset=' + str(25*i))
#         while not success:
#             comments_data = requests.get(url)
#             try:
#                 data = simplejson.loads(comments_data.content)
#                 success = True
#                 for d in data['results']['comments']:
#                     comments.append(d)
#                 time.sleep(2)
#             except:
#                 print 'error on {}'.format(str(da))
#                 print url
#                 time.sleep(2)
                
#     filestr = 'comments {}.json'.format(str(da))
#     with open(filestr, 'w') as f:
#         simplejson.dump(comments, f)

#  Exploratory Analysis

# Final Analysis